In [ ]:
x, y, z = Reals('x y z')
s = Solver()
s.add(x > 1, y > 1, x + y > 3, z - x < 10)
print(s.check())

m = s.model()
print("x = %s" % m[x])

print("traversing model...")
for d in m.decls():
    print("%s = %s" % (d.name(), m[d]))

In [11]:
x, y = Reals('x y')
solve(x + 10000000000000000000000 == y, y > 20000000000000000)

print(Sqrt(2) + Sqrt(3))
print(simplify(Sqrt(2) + Sqrt(3)))
print(simplify(Sqrt(2) + Sqrt(3)).sexpr())
# The sexpr() method is available for any Z3 expression
print((x + Sqrt(y) * 2).sexpr())



[y = 20000000000000001, x = -9999979999999999999999]
2**(1/2) + 3**(1/2)
3.1462643699?
(root-obj (+ (^ x 4) (* (- 10) (^ x 2)) 1) 4)
(+ x (* (^ y (/ 1.0 2.0)) 2.0))


# Funkcije

Za razliku od programskih jezika, gde funkcije imaju sporedne efekte, mogu da generišu izuzetke ili se nikada ne vraćaju, funkcije u Z3 nemaju neželjene efekte i potpune su. To jest, oni su definisani na svim ulaznim vrednostima. Ovo uključuje funkcije, kao što je podela. Z3 je zasnovan na logici prvog reda.

S obzirom na ograničenja kao što je k + i > 3, rekli smo da su k i i promenljive. U mnogim udžbenicima, k i i se nazivaju neprotumačenim konstantama. To jest, oni dozvoljavaju svako tumačenje koje je u skladu sa ograničenjem k + i > 3.

Tačnije, funkcijski i konstantni simboli u čistoj logici prvog reda su neinterpretirani ili slobodni, što znači da nije priloženo nikakvo apriorno tumačenje. Ovo je u suprotnosti sa funkcijama koje pripadaju potpisu teorija, kao što je aritmetika gde funkcija + ima fiksnu standardnu interpretaciju (dodaje dva broja). Neinterpretirane funkcije i konstante su maksimalno fleksibilne; dozvoljavaju svako tumačenje koje je u skladu sa ograničenjima funkcije ili konstante.

Da bismo ilustrovali neinterpretirane funkcije i konstante, hajde da koristimo neinterpretirane celobrojne konstante (aka promenljive) k, i. Konačno, neka f bude neinterpretirana funkcija koja uzima jedan argument tipa (aka sortiranje) celog broja i rezultira celobrojnom vrednošću. Primer ilustruje kako se može forsirati interpretacija gde f primenjen dvaput na k ponovo dovodi do k, ali f primenjen jednom na k se razlikuje od k.

In [12]:


x = Int('x')
y = Int('y')
f = Function('f', IntSort(), IntSort())
solve(f(f(x)) == x, f(x) == y, x != y)



[x = 0, y = 1, f = [1 -> 0, else -> 1]]


Rešenje (interpretacija) za f treba čitati kao f(0) je 1, f(1) je 0,

i f(a) je 1 za sve različite od 0 i 1.

In [13]:
x = Int('x')
y = Int('y')
f = Function('f', IntSort(), IntSort())
s = Solver()
s.add(f(f(x)) == x, f(x) == y, x != y)
print(s.check())
m = s.model()
print("f(f(x)) =", m.evaluate(f(f(x))))
print("f(x)    =", m.evaluate(f(x)))

sat
f(f(x)) = 0
f(x)    = 1


In [14]:
#In Z3, we can also evaluate expressions in the model for a system of constraints. The following example shows how to use the evaluate method.

x = Int('x')
y = Int('y')
f = Function('f', IntSort(), IntSort())
s = Solver()
s.add(f(f(x)) == x, f(x) == y, x != y)
print(s.check())
m = s.model()
print("f(f(x)) =", m.evaluate(f(f(x))))
print("f(x)    =", m.evaluate(f(x)))

sat
f(f(x)) = 0
f(x)    = 1


# Sudoku

In [2]:
from z3 import *


X = [ [ Int("x_%s_%s" % (i+1, j+1)) for j in range(9) ] 
      for i in range(9) ]


cells_c  = [ And(1 <= X[i][j], X[i][j] <= 9) 
             for i in range(9) for j in range(9) ]


rows_c   = [ Distinct(X[i]) for i in range(9) ]


cols_c   = [ Distinct([ X[i][j] for i in range(9) ]) 
             for j in range(9) ]

sq_c     = [ Distinct([ X[3*i0 + i][3*j0 + j] 
                        for i in range(3) for j in range(3) ]) 
             for i0 in range(3) for j0 in range(3) ]

sudoku_c = cells_c + rows_c + cols_c + sq_c

instance = ((0,0,0,0,9,4,0,3,0),
            (0,0,0,5,1,0,0,0,7),
            (0,8,9,0,0,0,0,4,0),
            (0,0,0,0,0,0,2,0,8),
            (0,6,0,2,0,1,0,5,0),
            (1,0,2,0,0,0,0,0,0),
            (0,7,0,0,0,0,5,2,0),
            (9,0,0,0,6,5,0,0,0),
            (0,4,0,9,7,0,0,0,0))

instance_c = [ If(instance[i][j] == 0, 
                  True, 
                  X[i][j] == instance[i][j]) 
               for i in range(9) for j in range(9) ]

s = Solver()
s.add(sudoku_c + instance_c)
if s.check() == sat:
    m = s.model()
    r = [ [ m.evaluate(X[i][j]) for j in range(9) ] 
          for i in range(9) ]
    print_matrix(r)
else:
    print("failed to solve")

[[7, 1, 5, 8, 9, 4, 6, 3, 2],
 [2, 3, 4, 5, 1, 6, 8, 9, 7],
 [6, 8, 9, 7, 2, 3, 1, 4, 5],
 [4, 9, 3, 6, 5, 7, 2, 1, 8],
 [8, 6, 7, 2, 3, 1, 9, 5, 4],
 [1, 5, 2, 4, 8, 9, 7, 6, 3],
 [3, 7, 6, 1, 4, 8, 5, 2, 9],
 [9, 2, 8, 3, 6, 5, 4, 7, 1],
 [5, 4, 1, 9, 7, 2, 3, 8, 6]]


# Predikatska logika

In [ ]:
# all humans are mortal
# Socrates is a human
# so Socrates mortal 
############################################

from z3 import *

Object = DeclareSort('Object')

Human = Function('Human', Object, BoolSort())
Mortal = Function('Mortal', Object, BoolSort())

# a well known philosopher 
socrates = Const('socrates', Object)

# free variables used in forall must be declared Const in python
x = Const('x', Object)

axioms = [ForAll([x], Implies(Human(x), Mortal(x))), 
          Human(socrates)]


s = Solver()
s.add(axioms)

print(s.check()) # prints sat so axioms are coherent

# classical refutation
s.add(Not(Mortal(socrates)))

print(s.check()) # prints unsat so socrates is Mortal

In [16]:
# Dve nemimoilazne prave se seku ili su paralelne.
# Prave koje se seku leže u istoj ravni.
# Prave koje su paralelene leže u istoj ravni.
# Dve nemimoilazne prave leže u istoj ravni.

# m(X,Y) - X i Y su nemimoilazne: m: PxP -> B
# s(X,Y) - X i Y se seku: s: PxP -> B
# p(X,Y) - X i Y su paralelne: p: PxP -> B
# r(X,Y) - X i Y leze u istoj ravni: r: PxP -> B

B = BoolSort()
P = DeclareSort('Prave')
m = Function('m', P, P, B)
s = Function('s', P, P, B)
p = Function('p', P, P, B)
r = Function('r', P, P, B)

x, y = Consts('x y', P)

solver = Solver()
axioms = [
    ForAll([x,y], Implies(m(x,y), Or(s(x,y), p(x,y)))),
    ForAll([x,y], Implies(s(x,y), r(x,y))),
    ForAll([x,y], Implies(p(x,y), r(x,y)))
]
conjecture = ForAll([x,y], Implies(m(x,y), r(x,y)))

solver.add(conjecture)

print(solver.check(axioms))



sat


In [17]:
# Svaka dva brata imaju zajedničkog roditelja.
# Roditelj je stariji od deteta.
# Postoje braća.
# Ni jedna osoba nije starija od druge.

# b: OxO -> B

B = BoolSort()
O = DeclareSort('Osoba')

b = Function('braca', O, O, B)
r = Function('roditelj', O, O, B)
s = Function('stariji', O, O, B)

x,y,z = Consts('x y z', O)

solver = Solver()
axioms = [
    ForAll([x,y], Exists([z], Implies(b(x,y), And(r(z,x), r(z,y))))),
    ForAll([x,y], Implies(r(x,y), s(x,y))),
    Exists([x,y], b(x,y))
]
conjecture = ForAll([x,y], Not(s(x,y)))

solver.add(conjecture)
solver.check(axioms)

unsat

# Uprošćavanje izraza

In [5]:
x = Int('x')
y = Int('y')
print(simplify(x + y + 2*x + 3))
print(simplify(x < y + x + 2))
print(simplify(And(x + 1 >= 3, x**2 + x**2 + y**2 + 2 >= 5))) 



3 + 3*x + y
Not(y <= -2)
And(x >= 2, 2*x**2 + y**2 >= 3)


# Šta Z3 ne može da uradi

Z3 deluje neverovatno moćno, i jeste. Međutim, pomoću Z3 rešavača možemo izraziti mnogo više stvari nego što on može da reši.

Na primer, faktorisanje celih brojeva je suština RSA kriptografije. Dok Z3 može faktorisati cele brojeve, ne može magično faktorisati čak ni umereno velike brojeve.

In [6]:
x,y = Ints("x y")
pubkey = 3	* 7
solve(x * y == pubkey, x > 1, y > 1) # easy peasy

[x = 7, y = 3]


In [7]:
x,y = Ints("x y")
pubkey = 1000000993	* 1000001011
solve(x * y == pubkey, x > 1, y > 1) # nope


failed to solve


Nelinearne jednačine će generalno biti teške. Nije nemoguće, ali teško. Z3 odmah odustaje ako pokušate da pronađete rešenje za naizgled rešiv problem koji uključuje eksponencijal. Z3 takođe ne može da razume sinuse, kosinuse i logaritme. Glavni izuzetak su ograničenja polinomske jednakosti, za koje z3 ima suštinsko razumevanje, međutim ove rutine mogu biti računski skupe.

In [ ]:
x = Real('x')
s = Solver()
s.add(2**x == 3)
print(s.check())


# Dokazati = Nema kontraprimera

Do sada smo koristili z3 da bismo pronašli rešenja za skup ograničenja/formula. Z3 je toliko efikasan u ovom procesu da može biti iscrpan i znati kada nema rešenja za pronalaženje. Ovo je korisno za dokazivanje teorema. Ako navedemo teoremu kao što je `Implies(And(p,k),p)` koja mora biti tačna za bilo koju vrednost p i k, možemo dokazati njenu istinitost tako što ćemo iscrpno tražiti kontraprimer i ne uspeti da ga pronađemo. Kontraprimer je dodeljivanje promenljivih za koje je interpretacija teoreme netačna, ili ekvivalentno za koje negacija Not(th) interpretira tačno.

solver.check() može da se vrati

 sat - "Našao sam rešenje. Možete ga tražiti pomoću solver.model()"
 nepoznato - Ovo znači "Odustajem". Možda negde postoji rešenje, a možda i ne.
 unsat - "ZNAM da nema rešenja. SVE sam probao".

Evo pojednostavljene (čak ni za toliko) verzije dokaza funkcije pogodnosti Z3 u kojoj kao dokaz izvodimo negaciju i interpretaciju nesatnih.


In [8]:
def prove2(f): # pojednostavljena verzija z3pi funkcije "dokaži"
    s = Solver()
    s.add(Not(f))
    if s.check() == unsat:
        print("proved")
    else:
        print("failed to prove")  

# Dokazivanje u iskaznoj logici

In [ ]:
p, q = Bools('p q')
print(And(p,q))
print(Or(p,q))
print(Xor(p,q))
print(Not(p))
print(Implies(p,q))
print(p == q)


my_true_thm = Implies(And(p,q), p)
prove(my_true_thm)


Za vežbu

Dokazati:

    De Morgan's Law p & q == ~ (~p | ~q)
    p -> q == ~ p | q
    Peirce's Law ((p -> q) -> p) -> p is always true



In [15]:
p, q = Bools('p q')
demorgan = And(p, q) == Not(Or(Not(p), Not(q)))
print(demorgan)

def prove(f):
    s = Solver()
    s.add(Not(f))
    if s.check() == unsat:
        print("proved")
    else:
        print("failed to prove")

print("Proving demorgan...")
prove(demorgan)



And(p, q) == Not(Or(Not(p), Not(q)))
Proving demorgan...
proved
